1. 使用完整的 YelpReviewFull 数据集训练，对比看 Acc 最高能到多少.

In [1]:
from datasets import load_dataset
dataset = load_dataset("yelp_review_full")


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


In [3]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [6]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

model_dir = "models/bert-base-cased"

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=5000,
                                  per_device_train_batch_size=32,
                                  num_train_epochs=1)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.703700,0.693340,0.694700


Checkpoint destination directory models/bert-base-cased/test_trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory models/bert-base-cased/test_trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=20313, training_loss=0.7641742852591802, metrics={'train_runtime': 18183.3478, 'train_samples_per_second': 35.747, 'train_steps_per_second': 1.117, 'total_flos': 1.710267926016e+17, 'train_loss': 0.7641742852591802, 'epoch': 1.0})

In [10]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.8430394530296326,
 'eval_accuracy': 0.67,
 'eval_runtime': 1.0255,
 'eval_samples_per_second': 97.515,
 'eval_steps_per_second': 12.677,
 'epoch': 1.0}

In [12]:
trainer.save_model(f"{model_dir}/finetuned-trainer")
trainer.save_state()

In [11]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.8430394530296326,
 'eval_accuracy': 0.67,
 'eval_runtime': 1.0347,
 'eval_samples_per_second': 96.646,
 'eval_steps_per_second': 12.564,
 'epoch': 1.0}